In [5]:
# Получившийся список должен содержать в себе минимум:
#
# 1) Наименование вакансии.
# 2) Предлагаемую зарплату (отдельно минимальную и максимальную; дополнительно - собрать валюту;
#    можно использовать regexp или if'ы).
# 3) Ссылку на саму вакансию.
# 4) Сайт, откуда собрана вакансия.
# По желанию можно добавить ещё параметры вакансии (например, работодателя и расположение).
# Структура должна быть одинаковая для вакансий с обоих сайтов. Сохраните результат в json-файл

In [6]:
import requests
from bs4 import BeautifulSoup
from dataclasses import dataclass
import pandas as pd
from time import sleep

In [24]:
# https://hh.ru/search/vacancy?area=1&fromSearchLine=true&text=Python&from=suggest_post&page=1&hhtmFrom=vacancy_search_list

@dataclass
class Vacancy:
    name = str
    domain = str
    salary = str
    min_salary = int
    max_salary = int
    currency = str
    site = str


    def get_salary_hh(self, salary_str):
        # self.salary = salary_str.replace('\u202f', '')
        self.currency = salary_str.split()[-1]
        if salary_str.find('–') != -1:
            self.min_salary = int(salary_str.replace('\u202f', '').split()[0])
            self.max_salary = int(salary_str.replace('\u202f', '').split()[2])
        elif salary_str.find('от') != -1:
            self.min_salary = int(salary_str.replace('\u202f', '').split()[1])
            self.max_salary = None
        elif salary_str.find('до') != -1:
            self.min_salary = None
            self.max_salary = int(salary_str.replace('\u202f', '').split()[1])
        else:
            self.min_salary = None


class VacancyList:
    headers = {
        'User-Agent': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (HTML, like Gecko) Chrome/97.0.4692.71 '
                      'Safari/537.36'}

    def __init__(self, vacancy_text, page=2):
        self.all_vacancy = []
        self.vacancy_text = vacancy_text
        self.page_count = page
        self.add_hh()

    def add_vacancy(self, vacancy):
        self.all_vacancy.append(vacancy)

    def add_hh(self):
        domain = 'https://hh.ru'
        url = '/search/vacancy'
        params_dict = {'clusters': 'true',
                       'area': 1,
                       'enable_snippets': 'true',
                       'st': 'searchVacancy',
                       'text': self.vacancy_text,
                       'page': 0}

        for page_count in range(0, self.page_count):
            params_dict['page'] = page_count
            response = requests.get(domain + url, params=params_dict, headers=self.headers)
            dom = BeautifulSoup(response.text, 'html.parser')

            card_list = dom.find_all('div', {'class': 'vacancy-serp-item'})

            for card in card_list:
                vacancy = Vacancy()
                vacancy.site = 'hh.ru'
                vacancy.name = card.find('a', {'data-qa': 'vacancy-serp__vacancy-title'}).text
                vacancy.domain = domain
                vacancy.link = card.find('a', {'data-qa': 'vacancy-serp__vacancy-title'})['href']

                try:
                    salary = card.find('span', {'data-qa': 'vacancy-serp__vacancy-compensation'}).text
                    vacancy.get_salary_hh(salary)

                except AttributeError:
                    vacancy.salary = None
                    vacancy.min_salary = None
                    vacancy.max_salary = None
                    vacancy.currency = None

                self.all_vacancy.append(vacancy)
            print(f'Обработка страницы {page_count + 1}')
            sleep(1)


final_list = VacancyList(vacancy_text='Data scientist', page=3)

vac_data = []
for vacancy in final_list.all_vacancy:
    vacancy_dict = {'name': vacancy.name,
                    # 'site': vacancy.site,
                    'link': vacancy.link,
                    'max_salary': vacancy.max_salary,
                    'min_salary': vacancy.min_salary,
                    'currency': vacancy.currency
                    }

    vac_data.append(vacancy_dict)

df = pd.DataFrame(vac_data)



Обработка страницы 1
Обработка страницы 2
Обработка страницы 3


In [25]:
df = pd.DataFrame(vac_data)
df

,name,link,max_salary,min_salary,currency
0,Data Scientist / специалист по распознаванию и...,https://samara.hh.ru/vacancy/53427577?from=vac...,NaN,100000.0,руб.
1,Senior Data scientist (Computer Vision),https://samara.hh.ru/vacancy/47357797?from=vac...,NaN,NaN,None
2,Data scientist,https://samara.hh.ru/vacancy/53215762?from=vac...,NaN,NaN,None
3,Senior Analytics & Insights Manager,https://samara.hh.ru/vacancy/51860512?from=vac...,NaN,NaN,None
4,Junior Data scientist/ Младший риск-аналитик,https://samara.hh.ru/vacancy/53211742?from=vac...,NaN,NaN,None
5,Data Scientist,https://samara.hh.ru/vacancy/53409941?from=vac...,5000.0,4000.0,EUR
6,Data scientist,https://samara.hh.ru/vacancy/52625579?from=vac...,220000.0,150000.0,руб.
7,Data scientist,https://samara.hh.ru/vacancy/53092406?from=vac...,NaN,300000.0,руб.
8,Data Scientist,https://samara.hh.ru/vacancy/49423067?from=vac...,320000.0,NaN,руб.
9,Data scientist / Product analyst,https://samara.hh.ru/vacancy/52276251?from=vac...,250000.0,200000.0,руб.


In [22]:
df.to_csv('./vacancy_data.csv', index=False)

In [23]:
%ls

get_salary_hh.py    homework_2.py       homework_2_2.py     vacancy_data.pkl
homework_2.ipynb    homework_2_2.ipynb  vacancy_data.csv
